In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import h5py

result_path = Path.home()/'group'/'project'/'results'/'beamforming'\
/'time_dependent'/'beamforming_grids'/'220421_dl_grid_84to90_1to45mm'

In [ ]:
for i, key in enumerate(np.load(result_path/f'220416_dl_grid_84to90_gradb_sum_rad1mm_0.npz')):
    print(key)

In [ ]:
x = np.load(result_path/f'220416_dl_grid_84to90_gradb_sum_rad1mm_0.npz')

print(x['signals'].shape)
print(x['radius'])
print(x['pitch'])
print(x['energy'])
print(x['freq'])

In [ ]:
index = 0

grid = abs(x['signals'][index, :, :]).sum(-1).reshape((7,7))

plt.imshow(grid)

In [ ]:
save_name = '220505_dl_grid_84to90deg_1to45mm.h5'
result_save_path = Path.home()/'group'/'project'/'results'/'beamforming'\
/'time_dependent'/'beamform_signals'

#combined_signals = np.zeros((33450, 8192), np.complex128)
combined_signals = []
combined_radius = []
combined_pitch = []
combined_energy = []
combined_freq = []
counter = 0

result_list = list(result_path.iterdir())

number_of_signals = 0
for i, item in enumerate(result_list):
    file_name = item.name
    chunk = np.load(result_path/item)
    shape = chunk['signals'].shape
    
    number_of_signals += shape[0]
    signal_length = shape[-1]
    
    if i % 10 == 9:
        print(f'{i + 1} / {len(result_list)}')
    
print(number_of_signals)

combined_shape = (number_of_signals, signal_length)

metakeys = ['radius', 'pitch', 'energy', 'grad-b-freq']
file = h5py.File(result_save_path/save_name, 'w')
file.create_dataset('x', combined_shape, dtype='c16')
for key in metakeys:
    file.create_dataset(key, (combined_shape[0],), dtype='f8' )

for i, item in enumerate(result_list):

    file_name = item.name
    chunk = np.load(result_path/item)
    
    signals = chunk['signals']
    radius = chunk['radius']
    pitch = chunk['pitch']
    energy = chunk['energy']
    freq = chunk['freq']

    signals_max = np.argmax(np.sum(abs(signals) ** 2, axis=-1), axis=-1).squeeze()

    for n in range(signals_max.size):
        file['x'][counter] = signals[n, signals_max[n], :]
        
        file['radius'][counter] = radius
        file['pitch'][counter] = pitch[n]
        file['energy'][counter] = energy[n]
        file['grad-b-freq'][counter] = freq[n]
        #combined_signals.append(signals[n, signals_max[n], :])
        #combined_radius.append(radius)
        #combined_pitch.append(pitch[n])
        #combined_energy.append(energy[n])
        #combined_freq.append(freq[n])
        counter += 1
        
    if counter % 100 == 99:
        print(counter + 1)
file.close()

In [ ]:
file.close()

In [ ]:
file.keys()

In [ ]:
print(np.unique(file['pitch'][:]))
print(np.unique(file['energy'][:]))
print(np.unique(file['radius']))

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(np.fft.fft(combined_signals[0]).real)

# save summed signal and rad, pitch info

In [ ]:
name = '220322_dl_grid_87deg_100eV_2cm_2mm_range'
result_path = Path.home()/'group'/'project'/'results'/'beamforming'\
/'time_dependent'/'beamform_signals'

combined_signals = np.array(combined_signals)

np.save(
    result_path/name,
    combined_signals
    #radius=combined_radius,
    #pitch=combined_pitch
)

In [ ]:
grid_shape = (51, 11)

In [ ]:
plt.imshow(np.mean(abs(array[0, :, :]) ** 2, axis=-1).reshape(grid_shape), aspect='auto')

# plot maps

In [ ]:
print(signals.shape)

In [ ]:
chunk_name = f'220310_dl_87deg_100ev_3cm_gradb_sum_rad24mm_0.npz'
chunk = np.load(result_path/chunk_name)
signals = chunk['signals']
#radius = chunk['radius']
#pitch = chunk['pitch']

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(1,1,1)
img = ax.imshow(np.mean(abs(signals[50, :, :]) ** 2, axis=-1).reshape((7, 7)), aspect='auto')
fig.colorbar(img)
plt.show()
plt.close()


In [ ]:
freq_path = Path.home()/'group'/'project'/'results'/'mayfly'/'211129_grad_b_frequency_grid_radius_angle.npz'

In [ ]:
frequencies, radii, pitch = np.load(freq_path)['freq'].flatten(), np.load(freq_path)['radii'].flatten(), np.load(freq_path)['angles'].flatten()

In [ ]:
radii, pitch = np.meshgrid(radii, pitch)
radii, pitch = radii.flatten(), pitch.flatten()

In [ ]:
print(radii[0:50], pitch[0:50])